In [1]:
from keras import applications
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils import layer_utils, np_utils
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras import backend as K

Using TensorFlow backend.


In [ ]:
image_width = 224
image_height = 224

# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
nb_train_samples = 1200 # number of all training samples belong to all possible classses
nb_validation_samples = 198 # number of all test samples belong to all possible classes
#train_dir = "D:/retinal_data_set_original/_1000_per_class_train/augmented_224/train"
train_dir = "D:/retinal_data_set_original/original/new_train"
#validation_dir = "D:/retinal_data_set_original/_1000_per_class_train/augmented_224/validation"
validation_dir = "D:/retinal_data_set_original/original/new_valid"
test_dir = "D:/retinal_data_set_original/original/new_test"

In [2]:
# Returns a compiled model identical to the previous one
loaded_model = load_model('D:/retinal_data_set_original/original/models/vgg16/vgg16_dr_last2.h5')

In [3]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_7 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               6422784   
_________________________________________________________________
batch_normalization_7 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 5)                 1285      
Total params: 21,139,781
Trainable params: 6,424,581
Non-trainable params: 14,715,200
________________________________________________________

In [4]:
intermediate_layer_model = Model(inputs=loaded_model.input,
                                 outputs=loaded_model.get_layer("dense_13").output)

In [5]:
intermediate_layer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_7 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               6422784   
Total params: 21,137,472
Trainable params: 6,422,784
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)

# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = intermediate_layer_model.predict_generator(generator, nb_train_samples // batch_size)
# save the output as a Numpy array
np.save(open('C:/Users/hp/Downloads/data/model/bottle_neck_features/bottleneck_features_train_vgg16.npy', 'wb'), bottleneck_features_train)

generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
bottleneck_features_validation = intermediate_layer_model.predict_generator(generator, nb_validation_samples // batch_size)
np.save(open('C:/Users/hp/Downloads/data/model/bottle_neck_features/bottleneck_features_validation_vgg16.npy', 'wb'), bottleneck_features_validation)

In [6]:
# load an image and preprocess it
img_path = 'D:/retinal_data_set_original/original/new_test/4/pdr_0_1617478.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# get the features 
extract_features = intermediate_layer_model.predict(x)

In [7]:
extract_features

array([[  0.       ,   0.       ,   0.       , 217.91887  ,  26.607054 ,
        651.4683   ,   0.       , 568.51624  ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         72.3665   ,   0.       ,   0.       ,   0.       ,   1.2301357,
          1.4975735,   0.       ,   0.       , 166.5935   ,  69.04588  ,
          0.       , 403.3317   ,  65.23279  ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       , 409.04227  , 497.1649   ,
          0.       ,   0.       , 109.632744 ,   0.       ,   0.       ,
        442.20618  ,   0.       ,   7.276459 ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,  22.980484 ,
          0.       ,   0.       , 122.70509  , 651.7275   ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       , 359.47452  ,   0.       ,
        456.59915  ,  22.703203 ,   0.       ,   0.

In [8]:
get_3rd_layer_output = K.function([loaded_model.get_input_at(0), K.learning_phase()],
                              [loaded_model.get_layer("dense_13").output])


In [9]:
# output in test mode = 0
layer_output = get_3rd_layer_output([x, 0])[0]

In [10]:
layer_output

array([[  0.       ,   0.       ,   0.       , 217.91887  ,  26.607054 ,
        651.4683   ,   0.       , 568.51624  ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         72.3665   ,   0.       ,   0.       ,   0.       ,   1.2301357,
          1.4975735,   0.       ,   0.       , 166.5935   ,  69.04588  ,
          0.       , 403.3317   ,  65.23279  ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       , 409.04227  , 497.1649   ,
          0.       ,   0.       , 109.632744 ,   0.       ,   0.       ,
        442.20618  ,   0.       ,   7.276459 ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,  22.980484 ,
          0.       ,   0.       , 122.70509  , 651.7275   ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       , 359.47452  ,   0.       ,
        456.59915  ,  22.703203 ,   0.       ,   0.